# Prototype
[*v0.1*]

Prototyping with TMDB.

# Setup

Cells in this section handle notebook setup, like importing packages and functions/vars from scripts.

## Imports

Import `stdlib` packages (i.e. `pathlib.Path`) and package dependencies.

### stdlib

In [1]:
from pathlib import Path
import json
from typing import Any, Optional, Union
import random

### Custom modules & vars

In [2]:
## Constants
from lib.constants import (
    auth_endpoint,
    movie_endpoint,
    token_endpoint,
    api_key,
    base_url,
    tv_endpoint,
    session_endpoint,
    popular_tv_endpoint,
    basic_auth_headers,
)

In [3]:
from utils.time_utils import benchmark
from utils.file_utils import check_file_exist

In [4]:
from core.config import api_settings, app_settings, logging_settings
from utils.logger import get_logger

In [5]:
from utils.tmdb_utils import (
    authenticate,
    get_popular_tv,
    get_request_token,
    retrieve_token,
    get_bad_ids,
    append_bad_id,
    check_bad_id,
    generate_rand_id,
    get_tv_episode,
)

In [6]:
from core.db import Base, create_base_metadata, get_engine, get_session
from domain.schemas.tmdb import tmdb_media_schemas, tmdb_responses

### Dependencies

Packages installed with `pip` (or some equivalent tool)

In [7]:
import httpx

## Global Vars

Variables for use throughout the notebook

In [8]:
nb_log: bool = True
nb_verbose: bool = False

In [9]:
log = get_logger(__name__, level="INFO")

In [10]:
engine = get_engine(connection="db/demo.sqlite", echo=True)
SessionLocal = get_session(engine=engine)
create_base_metadata(base_obj=Base, engine=engine)

2023-06-19 17:32:46,901 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-19 17:32:46,902 INFO sqlalchemy.engine.Engine COMMIT


True

## Functions

Notebook-level functions. These differ from functions imported from scripts in that they are either prototypes, or functions meant only for the notebook.

### Notebook Functions

### Prototype

## Classes

Notebook-level classes. These differ from classes/models imported from scripts in that they are either prototypes, or functions meant only for the notebook.

# Operations

Functions & data operations.

In [11]:
## Test authentication
_auth = authenticate()
display(f"Auth response: [{_auth.status_code}: {_auth.reason_phrase}]")

[INFO][2023-06-19_17:32:46][utils.tmdb_utils][authenticate ln: 37]: Requesting https://api.themoviedb.org/3/authentication


'Auth response: [200: OK]'

In [12]:
## Test retrieving auth token
_token = get_request_token()
display(f"Token response: [{_token.status_code}: {_auth.reason_phrase}]")

## Retrieve token string from resonse
token = retrieve_token(_token.text)
display(f"Token ({type(token)}): {token}")

[INFO][2023-06-19_17:32:47][utils.tmdb_utils][get_request_token ln: 91]: Requesting https://api.themoviedb.org/3/authentication/token/new


'Token response: [200: OK]'

"Token (<class 'domain.schemas.tmdb.tmdb_responses.ReqToken'>): success=True expires_at='2023-06-19 22:32:47 UTC' request_token='b6e6f8d4164ffcab82f99c5b30b31bd9875c0cba'"

## Examples

### Example responses

In [13]:
examples_dir: str = "examples"
example_responses_dir: str = "responses"
ex_res_path: str = f"{examples_dir}/{example_responses_dir}"

if not Path(ex_res_path).exists():
    Path(ex_res_path).mkdir(exist_ok=True, parents=True)

In [14]:
ex_movie_res_file: str = f"{ex_res_path}/ex_movie_response.json"
ex_tv_res_file: str = f"{ex_res_path}/ex_tvshow_response.json"

In [15]:
with open(ex_movie_res_file, "r+") as movie_res_file:
    ex_movie_res: dict = json.loads(movie_res_file.read())

with open(ex_tv_res_file, "r+") as tv_res_file:
    ex_tv_res: dict = json.loads(tv_res_file.read())

In [16]:
ex_movie_keys = ex_movie_res.keys()
# display(f"Movie dict keys:")

# for k in ex_movie_keys:
#     display(f"Key [{k}] ({type(ex_movie_res[k]).__name__}): {ex_movie_res[k]}")

In [17]:
ex_tv_keys = ex_tv_res.keys()
# display(f"TV dict keys:")

# for k in ex_tv_keys:

In [18]:
shared_keys: list[str] = []
movie_only_keys: list[str] = []
tv_only_keys: list[str] = []

In [19]:
for k in ex_movie_keys:
    if not k in ex_tv_keys:
        # display(f"Movie-specific key found: {k} ({type(ex_movie_res[k]).__name__})")
        movie_only_keys.append(k)

    else:
        if not k in shared_keys:
            shared_keys.append(k)

In [20]:
for k in ex_tv_keys:
    if not k in ex_movie_keys:
        # display(f"TV-specific key found: {k} ({type(ex_tv_res[k]).__name__})")

        tv_only_keys.append(k)

    else:
        if not k in shared_keys:
            shared_keys.append(k)

In [21]:
display(f"Keys only in movie response:")

for movie_k in movie_only_keys:
    display(movie_k)

'Keys only in movie response:'

'belongs_to_collection'

'budget'

'imdb_id'

'original_title'

'release_date'

'revenue'

'runtime'

'title'

'video'

In [22]:
display(f"Keys only in tv response:")

for tv_k in tv_only_keys:
    display(tv_k)

'Keys only in tv response:'

'created_by'

'episode_run_time'

'first_air_date'

'in_production'

...

'number_of_seasons'

'origin_country'

'original_name'

'seasons'

'type'

In [23]:
display(f"Keys in both movies and tv shows:")

display(shared_keys)

'Keys in both movies and tv shows:'

['adult',
 'backdrop_path',
 'genres',
 'homepage',
 'id',
 'original_language',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'spoken_languages',
 'status',
 'tagline',
 'vote_average',
 'vote_count']

### TV Show examples

In [24]:
## Retrieve popular TV shows
popular_tv = get_popular_tv()
display(
    f"Retrieve popular TV shows response: [{popular_tv.status_code}: {popular_tv.reason_phrase}]"
)

[INFO][2023-06-19_17:32:47][utils.tmdb_utils][get_popular_tv ln: 223]: Requesting https://api.themoviedb.org/3/tv/popular?language=en-US&page=1


'Retrieve popular TV shows response: [200: OK]'

In [25]:
## Create dictionary from popular_tv response
pop_tv_dict: dict = popular_tv.text_json()
display(pop_tv_dict.keys())

display(pop_tv_dict["results"][0])

display(f"genre_ids ({type(pop_tv_dict['results'][0]['genre_ids'])})")

## Create MediaResponse class instance from pop_tv_dict
pop_tv: tmdb_media_schemas.MediaResponse = tmdb_media_schemas.MediaResponse.parse_obj(
    pop_tv_dict
)

# display(f"Popular TV shows response type: ({type(pop_tv)}): {pop_tv}")

dict_keys(['page', 'results', 'total_pages', 'total_results'])

{'backdrop_path': '/t2rAdgjSh0WYbXzdOB5zTDqzdCI.jpg',
 'first_air_date': '2022-11-02',
 'genre_ids': [18],
 'id': 213713,
 'name': 'Faltu',
 'origin_country': ['IN'],
 'original_language': 'hi',
 'original_name': 'Faltu',
 'overview': "What's in a name? Amidst the arid landscape of Rajasthan, a young woman with dreamy eyes struggles to prove her worth.",
 'popularity': 2699.206,
 'poster_path': '/lgyFuoXs7GvKJN0mNm7z7OMOFuZ.jpg',
 'vote_average': 4.6,
 'vote_count': 29}

"genre_ids (<class 'list'>)"

In [26]:
## grab a random sample TV show from popular TV shows
pop_tv_results: int = len(pop_tv.results)
rand_pop_tv: int = random.randint(0, pop_tv_results - 1)

_sample: tmdb_media_schemas.MediaTVShow = pop_tv.results[rand_pop_tv]
display(f"Sample ({type(_sample)}): {_sample}")

'Sample (<class \'domain.schemas.tmdb.tmdb_media_schemas.MediaTVShow\'>): adult=None backdrop_path=\'/jJy6BwcEYphxGn0SGB30l98Jvje.jpg\' genres=None genre_ids=[10764] overview="MasterChef Australia is a Logie Award-winning Australian competitive cooking game show based on the original British MasterChef. It is produced by Shine Australia and screens on Network Ten. Restaurateur and chef Gary Mehigan, chef George Calombaris and food critic Matt Preston serve as the show\'s main judges. Journalist Sarah Wilson hosted the first series, however her role was dropped at the end of the series." popularity=1444.701 poster_path=\'/m5akdtbWznF8KpOewKyKw0C36s1.jpg\' tmdb_id=16395 original_language=\'en\' vote_average=7.0 vote_count=64 homepage=None production_companies=None runtime=None created_by=None episode_run_time=None first_air_date=\'2009-04-27\' in_production=None languages=None last_air_date=None last_episode_to_air=None next_episode_to_air=None name=\'MasterChef Australia\' origin_countr

In [27]:
## Retrieve full listing from TMDB
url = f"{api_settings.BASE_URL}/tv/{_sample.tmdb_id}"
display(f"Sample TV URL: {url}")

'Sample TV URL: https://api.themoviedb.org/3/tv/16395'

In [28]:
_show = get_tv_episode(tmdb_id=_sample.tmdb_id)
_show_dict: dict = _show.text_json()

tv_show: tmdb_media_schemas.MediaTVShow = tmdb_media_schemas.MediaTVShow.parse_obj(
    _show_dict
)

[INFO][2023-06-19_17:32:47][utils.tmdb_utils][get_tv_episode ln: 275]: Requesting https://api.themoviedb.org/3/tv/16395


In [29]:
display(f"Sample TV Show ({type(tv_show).__name__})")

display(tv_show)

'Sample TV Show (MediaTVShow)'

MediaTVShow(adult=False, backdrop_path='/t8dpSMqwCu6hTxFE85lZyc5l52o.jpg', genres=[MediaGenres(id=10764, name='Reality')], genre_ids=None, overview="MasterChef Australia is a Logie Award-winning Australian competitive cooking game show based on the original British MasterChef. It is produced by Shine Australia and screens on Network Ten. Restaurateur and chef Gary Mehigan, chef George Calombaris and food critic Matt Preston serve as the show's main judges. Journalist Sarah Wilson hosted the first series, however her role was dropped at the end of the series.", popularity=1444.701, poster_path='/m5akdtbWznF8KpOewKyKw0C36s1.jpg', tmdb_id=16395, original_language='en', vote_average=6.908, vote_count=65, homepage='https://10play.com.au/masterchef', production_companies=[ProductionCompanies(id=24645, logo_path='/9XMHkEXi5H6w5b1NOLEgo7tLK5F.png', name='EndemolShine Australia', origin_country='AU')], runtime=None, created_by=[TVShowCreator(id=65304, credit_id='52584e33760ee34661021c20', name=

### Random movie

In [30]:
movie_objs: list[tmdb_media_schemas.MediaMovie] = []
rand_movie = generate_rand_id(type="movie")

display(f"Random movie ID: {rand_movie}")

'Random movie ID: 78'

In [31]:
url = f"https://api.themoviedb.org/3/movie/{rand_movie}?language=en-US"
display(f"URL: {url}")

'URL: https://api.themoviedb.org/3/movie/78?language=en-US'

In [32]:
## Get random movie
try:
    with httpx.Client(headers=basic_auth_headers) as client:
        res = client.get(url)

        if not res.status_code == 200:
            display(
                f"Non-200 response returned: [{res.status_code}: {res.reason_phrase}]: {res.text}"
            )

            display(f"Adding ID {rand_movie} to list of known bad IDs")

            append_bad_id(bad_id=rand_movie, bad_id_file="bad_movie_ids")

        results: tmdb_responses.ReqResponse = tmdb_responses.ReqResponse.parse_obj(
            res.__dict__
        )

        res_json: dict = json.loads(res.text)

        _movie: tmdb_media_schemas.MediaMovie = tmdb_media_schemas.MediaMovie.parse_obj(
            res_json
        )

        display(f"Movie: {_movie}")

        movie_objs.append(_movie)

except Exception as exc:
    raise Exception(
        f"Unhandled exception requesting movie with ID: {rand_movie}. Details: {exc}"
    )

"Movie: adult=False backdrop_path='/eIi3klFf7mp3oL5EEF4mLIDs26r.jpg' genres=[MediaGenres(id=878, name='Science Fiction'), MediaGenres(id=18, name='Drama'), MediaGenres(id=53, name='Thriller')] genre_ids=None overview='In the smog-choked dystopian Los Angeles of 2019, blade runner Rick Deckard is called out of retirement to terminate a quartet of replicants who have escaped to Earth seeking their creator for a way to extend their short life spans.' popularity=60.351 poster_path='/63N9uy8nd9j7Eog2axPQ8lbr3Wj.jpg' tmdb_id=78 original_language='en' vote_average=7.929 vote_count=12486 homepage='http://www.warnerbros.com/blade-runner' production_companies=[ProductionCompanies(id=5798, logo_path='/tmuI9BGXgpWLmokhlxpnG3IGNQB.png', name='Shaw Brothers', origin_country='HK'), ProductionCompanies(id=7965, logo_path=None, name='The Ladd Company', origin_country='US'), ProductionCompanies(id=174, logo_path='/IuAlhI9eVC9Z8UQWOIDdWRKSEJ.png', name='Warner Bros. Pictures', origin_country='US')] runti

In [33]:
display(movie_objs)

[MediaMovie(adult=False, backdrop_path='/eIi3klFf7mp3oL5EEF4mLIDs26r.jpg', genres=[MediaGenres(id=878, name='Science Fiction'), MediaGenres(id=18, name='Drama'), MediaGenres(id=53, name='Thriller')], genre_ids=None, overview='In the smog-choked dystopian Los Angeles of 2019, blade runner Rick Deckard is called out of retirement to terminate a quartet of replicants who have escaped to Earth seeking their creator for a way to extend their short life spans.', popularity=60.351, poster_path='/63N9uy8nd9j7Eog2axPQ8lbr3Wj.jpg', tmdb_id=78, original_language='en', vote_average=7.929, vote_count=12486, homepage='http://www.warnerbros.com/blade-runner', production_companies=[ProductionCompanies(id=5798, logo_path='/tmuI9BGXgpWLmokhlxpnG3IGNQB.png', name='Shaw Brothers', origin_country='HK'), ProductionCompanies(id=7965, logo_path=None, name='The Ladd Company', origin_country='US'), ProductionCompanies(id=174, logo_path='/IuAlhI9eVC9Z8UQWOIDdWRKSEJ.png', name='Warner Bros. Pictures', origin_coun